In [104]:
import os
import sys
import pandas as pd
import numpy as np
import locale
locale.setlocale(locale.LC_ALL, '')

def generate_report(output_dev):
    output_dev.write("Financial Analysis\n------------------ \n")
    output_dev.write("Total Months: " + str(summary["Total Months"]) + "\n")
    output_dev.write("Total: " + locale.currency(summary["Total"], grouping=True) + "\n")
    output_dev.write("Average Change: " + locale.currency(summary["Average Change"], grouping=True) + "\n")
    output_dev.write("Greatest Increase in Profits: " + summary["Max Increase"][0] + " (" + locale.currency(summary["Max Increase"][1], grouping=True) + ")\n")
    output_dev.write("Greatest Decrease in Profits: " + summary["Max Decrease"][0] + " (" + locale.currency(summary["Max Decrease"][1], grouping=True) + ")\n")

# get the data and load it into a data frame
my_dir = os.getcwd()
data_file = os.path.join(my_dir, "Resources/budget_data.csv")

bank_df = pd.read_csv(data_file, encoding="utf-8")


      Date  Profit/Losses
0   Jan-10        1088983
1   Feb-10        -354534
2   Mar-10         276622
3   Apr-10        -728133
4   May-10         852993
..     ...            ...
81  Oct-16        -729004
82  Nov-16        -112209
83  Dec-16         516313
84  Jan-17         607208
85  Feb-17         382539

[86 rows x 2 columns]


In [106]:
# create a monthly change column in the bank dataframe
# while we're looping, find max and min change information
delta = np.array([float("nan")])
delta_max = 0
delta_min = 0
for i in range(1,len(bank_df)):
    delta = np.append(delta, (bank_df["Profit/Losses"][i] - bank_df["Profit/Losses"][i-1]))
    if delta[i] > delta_max:
        delta_max = delta[i]
        month_max = bank_df["Date"][i]
    if delta[i] < delta_min:
        delta_min = delta[i]
        month_min = bank_df["Date"][i]
bank_df["Monthly Change"] = delta.tolist()


      Date  Profit/Losses  Monthly Change
0   Jan-10        1088983             NaN
1   Feb-10        -354534      -1443517.0
2   Mar-10         276622        631156.0
3   Apr-10        -728133      -1004755.0
4   May-10         852993       1581126.0
..     ...            ...             ...
81  Oct-16        -729004      -1627245.0
82  Nov-16        -112209        616795.0
83  Dec-16         516313        628522.0
84  Jan-17         607208         90895.0
85  Feb-17         382539       -224669.0

[86 rows x 3 columns]


In [107]:
# generate summary data
summary = dict()
length = len(bank_df["Date"])
summary["Total Months"] = length
summary["Total"] = bank_df["Profit/Losses"].sum()
summary["Average Change"] = bank_df["Monthly Change"].loc[1:length-1].mean()
summary["Max Increase"] = [month_max, delta_max]
summary["Max Decrease"] = [month_min, delta_min]


In [108]:
# write report to the terminal
generate_report(sys.stdout)

Financial Analysis
------------------ 
Total Months: 86
Total: $22,564,198.00
Average Change: -$8,311.11
Greatest Increase in Profits: Aug-16 ($1,862,002.00)
Greatest Decrease in Profits: Feb-14 (-$1,825,558.00)


In [109]:
# write report to an output file
output_file = os.path.join(my_dir, "analysis/summary.txt")
with open(output_file, "w") as f:
    generate_report(f)
    f.close()